In [1]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
from collections import OrderedDict
import os

# Import necessary libraries for data fetching
from quantderivstrat.vendors.bloomberg.api import API

def create_excel_data():
    # Define the instruments and their tickers
    instruments_data = pd.DataFrame({
        'Ticker': [
            'GACGB2 Index', 'GACGB10 Index', 'TUAFWD Comdty', 'FVAFWD Comdty',
            'TYAFWD Comdty', 'UXYAFWD Comdty', 'WNAFWD Comdty', 'DUAFWD Comdty',
            'OEAFWD Comdty', 'RXAFWD Comdty', 'GAFWD Comdty', 'IKAFWD Comdty',
            'CNAFWD Comdty', 'JBAFWD Comdty', 'CCSWNI1 Curncy', 'ADSW2 Curncy',
            'CDSO2 Curncy', 'USSW2 Curncy', 'EUSA2 Curncy', 'BPSWS2 BGN Curncy',
            'NDSWAP2 BGN Curncy', 'I39302Y Index', 'MPSW2B BGN Curncy',
            'MPSWF2B Curncy', 'SAFR1I2 BGN Curncy', 'CKSW2 BGN Curncy',
            'PZSW2 BGN Curncy', 'KWSWNI2 BGN Curncy', 'CCSWNI2 CMPN Curncy',
            'ADSW5 Curncy', 'CDSO5 Curncy', 'USSW5 Curncy', 'EUSA5 Curncy',
            'BPSWS5 BGN Curncy', 'NDSWAP5 BGN Curncy', 'I39305Y Index',
            'MPSW5E Curncy', 'MPSWF5E Curncy', 'SASW5 Curncy', 'CKSW5 Curncy',
            'PZSW5 Curncy', 'KWSWNI5 Curncy', 'CCSWNI5 Curncy', 'JYSO5 Curncy',
            'ADSW10 Curncy', 'CDSO10 Curncy', 'USSW10 Curncy', 'EUSA10 Curncy',
            'BPSWS10 BGN Curncy', 'NDSWAP10 BGN Curncy', 'ADSW30 Curncy',
            'CDSW30 Curncy', 'USSW30 Curncy', 'EUSA30 Curncy', 'BPSWS30 BGN Curncy',
            'NDSWAP30 BGN Curncy', 'JYSO30 Curncy', 'MPSW10J BGN Curncy',
            'MPSWF10J BGN Curncy', 'SASW10 Curncy', 'CKSW10 Curncy',
            'PZSW10 Curncy', 'KWSWNI10 Curncy', 'CCSWNI10 Curncy', 'BPSWIT10 Curncy', 'SKSW10 Curncy', 'NKSW10 Curncy'
        ],
        'Instrument Name': [
            'AU 3Y Future', 'AU 10Y Future', 'US 2Y Future', 'US 5Y Future',
            'US 10Y Future', 'US 10Y Ultra Future', 'US 30Y Future', 'DE 2Y Future',
            'DE 5Y Future', 'DE 10Y Future', 'UK 10Y Future', 'IT 10Y Future',
            'CA 10Y Future', 'JP 10Y Future', 'CH 1Y Swap', 'AU 2Y Swap',
            'CA 2Y Swap', 'US 2Y Swap', 'DE 2Y Swap', 'UK 2Y Swap',
            'NZ 2Y Swap', 'BR 2Y Swap', 'MX 2Y Swap', 'MX 2Y Swap OIS',
            'SA 2Y Swap', 'CZ 2Y Swap', 'PO 2Y Swap', 'SK 2Y Swap',
            'CH 2Y Swap', 'AU 5Y Swap', 'CA 5Y Swap', 'US 5Y Swap', 'DE 5Y Swap',
            'UK 5Y Swap', 'NZ 5Y Swap', 'BR 5Y Swap', 'MX 5Y Swap',
            'MX 5Y Swap OIS', 'SA 5Y Swap', 'CZ 5Y Swap', 'PO 5Y Swap',
            'SK 5Y Swap', 'CH 5Y Swap', 'JP 5Y Swap', 'AU 10Y Swap',
            'CA 10Y Swap', 'US 10Y Swap', 'DE 10Y Swap', 'UK 10Y Swap',
            'NZ 10Y Swap', 'AU 30Y Swap', 'CA 30Y Swap', 'US 30Y Swap',
            'DE 30Y Swap', 'UK 30Y Swap', 'NZ 30Y Swap', 'JP 30Y Swap',
            'MX 10Y Swap', 'MX 10Y Swap OIS', 'SA 10Y Swap', 'CZ 10Y Swap',
            'PO 10Y Swap', 'SK 10Y Swap', 'CH 10Y Swap', 'UK 10Y Swap Inf', 'SW 10Y Swap', "NK 10Y Swap"
        ]
    })

    # Create tickers_data mapping Ticker to Instrument Name
    tickers_data = OrderedDict(zip(instruments_data['Ticker'], instruments_data['Instrument Name']))

    # Set start and end dates for data fetching
    end_date = datetime.today()
    start_date = end_date - timedelta(days=2520 + 252)  # Fetch extra data for calculations

    # Define tickers and their names
    tickers = list(tickers_data.keys())

    # Initialize Bloomberg API
    api = API(debug=False, port=8194, timeout=10000)

    # Fetch historical prices
    prices = api.bdh(
        tickers=tickers,
        fields=['px_last'],
        elements=[
            ("periodicityAdjustment", "CALENDAR"),
            ("periodicitySelection", "DAILY"),
            ("nonTradingDayFillMethod", "PREVIOUS_VALUE"),
            ("nonTradingDayFillOption", "ALL_CALENDAR_DAYS")
        ],
        sdate=start_date,
        edate=end_date
    )

    # Format the DataFrame
    df = pd.pivot_table(prices, index='date', values='px_last', columns='ticker')

    # Rename columns for clarity
    df.rename(columns=tickers_data, inplace=True)

    # Define the path to save the Excel file
    excel_file_path = r'C:\Users\MFegan\OneDrive - AustralianSuper PTY LTD\Documents\GitHub\riskapp\historical_data.xlsx'

    # Save the DataFrame to the specified Excel file
    df.to_excel(excel_file_path)

    print(f"Historical data has been saved to '{excel_file_path}'.")

if __name__ == '__main__':
    create_excel_data()

# Print the current working directory and file path
print("Excel file has been saved to:", r'C:\Users\MFegan\OneDrive - AustralianSuper PTY LTD\Documents\GitHub\riskapp\historical_data.xlsx')
print("Current Working Directory:", os.getcwd())


<ipython-input-1-fd704650b8af>:90: UserWarning: Pandas requires version '1.4.3' or newer of 'xlsxwriter' (version '1.3.8' currently installed).
  df.to_excel(excel_file_path)


Historical data has been saved to 'C:\Users\MFegan\OneDrive - AustralianSuper PTY LTD\Documents\GitHub\riskapp\historical_data.xlsx'.
Excel file has been saved to: C:\Users\MFegan\OneDrive - AustralianSuper PTY LTD\Documents\GitHub\riskapp\historical_data.xlsx
Current Working Directory: c:\Users\MFegan\OneDrive - AustralianSuper PTY LTD\Documents\GitHub\riskapp
